# Imports

In [54]:
import socket # tcp ip client
import time   # time measurement
import select # timeout tcp ip client recv

# Open TCP port

In [59]:
ip_address = '1.2.3.4'
port = 2000

buffer_size = 64

s=socket.socket(socket.AF_INET, socket.SOCK_STREAM)
#s.setblocking(0)  # set blocking so recv is never blocked
s.connect((ip_address, port))


# Run test cases and close connection

In [60]:
messages  = ['[SIGN_IN%a]',
             '[ORDER%ilias2017%6;1;00244;10342;61130;30534;51221;21413]',
             '[SIGN_IN%ilias2017]', 
             '[SIGN_IN%ilias2017]', 
             '[ORDER%ilias2017%6;1;00244;10342;61130;30534;51221;21413]',
             '[ORDER%a%1;2;11111;22222;33333]',
             '[ORDER%ilias2017%6;1;00244;10342;61130;30534;51221;21413]',
             '[ORDER%ilias2017%6;1;00244;10342;61130;30534;51221;21413]',
             '[ORDER%ilias2017%6;1;00244;10342;61130;30534;51221;21413]',
             '[ORDER%ilias2017%6;1;00244;10342;61130;30534;51221;21413]',
             '[SIGN_OUT]'
            ]

messages2 = ['[SIGN_IN%a]', # sign in wrong
             '[ORDER%ilias2017%20;1;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244]', #order before sign in
             '[SIGN_IN%ilias2017]', # sign in correct
             '[ORDER%ilias2017%20;1;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244]', # first roder
             '[SIGN_IN%ilias2017]', # keep alive
             '[SIGN_OUT]'
            ]

messages3  = ['[SIGN_IN%a]',
            '[SIGN_IN%ilias2017]', 
            '[SIGN_IN%ilias2017]', 
            '[ORDER%ilias2017%6;1;00244;10342;61130;30534;51221;21413]',
            '[ORDER%a%1;2;11111;22222;33333]',
            '[SIGN_OUT]'
            ]

active_message_list = messages2

# read initial data 
ready = select.select([s],[],[], 3 )
if ready[0]:
    data = s.recv(buffer_size)
else:
    data = "no data received"
print('Connect: <- ' + data)
    

for index, message in enumerate( active_message_list):
    s.send(active_message_list[ index ])

    ready = select.select([s],[],[], 4 )
    time.sleep(3)
    if ready[0]:
        data = s.recv(buffer_size)
    else:
        data = "no data received"
    print( 'TC' + str(index) + ': -> ' + message + ', <- ' + data )
    time.sleep(1)
s.close()
print( "Connection closed!")

Connect: <- *HELLO*
TC0: -> [SIGN_IN%a], <- [SIGN_IN_RS%0%0%PW_WRONG]

TC1: -> [ORDER%ilias2017%20;1;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244], <- no data received
TC2: -> [SIGN_IN%ilias2017], <- [ORDER_RS%ORDER_WRONG%0%0]

TC3: -> [ORDER%ilias2017%20;1;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244], <- no data received
TC4: -> [SIGN_IN%ilias2017], <- [ORDER_RS%ORDER_WRONG%0%0]

TC5: -> [SIGN_OUT], <- no data received
Connection closed!


# Single tests

In [68]:
message = '[SIGN_OUT]'

s.send(message)
time.sleep(3)
data = s.recv(buffer_size)
print( data)

error: [Errno 10053] An established connection was aborted by the software in your host machine

In [19]:
s.close()